In [1]:
import numpy as np
import astropy as ast
import matplotlib.pyplot as plt

from astropy import units as u

In [2]:
def get_phi(vector: np.ndarray) -> float:
    """Return the azimuthal angle of vector (in conventional spherical coordinates).
    """
    
    if vector[0] > 0:
        return np.arctan(vector[1] / vector[0])
    elif vector[0] < 0 :
        return np.arctan(vector[1] / vector[0]) + np.pi
    else:
        if vector[1] > 0:
            return np.pi / 2
        elif vector[1] < 0:
            return -np.pi / 2
        else:
            return 0

In [3]:
def get_theta(vector: np.ndarray) -> float:
    """Return the polar angle of vector (in conventional spherical coordinates).
    """
    
    rho = np.sqrt(np.sum(vector ** 2))
    
    if rho != 0:
        return np.arccos(vector[2] / rho)
    else:
        return np.pi / 2

In [4]:
def build_triple_rotation_matrix(ref_r: np.ndarray, ref_L: np.ndarray) -> np.ndarray:
    """Return a matrix object to transform vectors to from an original coordinate system
    to a coordinate system where (x = ref_r, y = 0, z = 0) and z points along ref_L.
    """
    
    # get the angles of spherical coordinates of ref_r
    phi = get_phi(ref_r)
    theta = get_theta(ref_r)
    
    # rotation 1 rotates around z-axis 
    rotation_1 = np.zeros([3, 3])
    
    rotation_1[0][0] = np.cos(phi)
    rotation_1[0][1] = np.sin(phi)
    rotation_1[1][0] = -np.sin(phi)
    rotation_1[1][1] = np.cos(phi)
    rotation_1[2][2] = 1
    
    # rotation 2 rotates around new y-axis
    rotation_2 = np.zeros([3, 3])
    
    rotation_2[0][0] = np.sin(theta)
    rotation_2[0][2] = np.cos(theta)
    rotation_2[2][0] = -np.cos(theta)
    rotation_2[2][2] = np.sin(theta)
    rotation_2[1][1] = 1
    
    double_rotation = rotation_2 @ rotation_1
    
    # rotate ref_L by rotation_1 then rotation_2
    interm_L = double_rotation @ ref_L
    
    # rearrange components of interm_L to get azimuthal angle w.r.t. x instead of z
    pseudo_L = np.array([
        interm_L[1],
        interm_L[2],
        interm_L[0]
    ])
    
    pseudo_phi = get_phi(pseudo_L)
    
    # rotation_3 rotates around new x-axis
    rotation_3 = np.zeros([3, 3])
    
    rotation_3[0][0] = 1
    rotation_3[1][1] = np.sin(pseudo_phi)
    rotation_3[1][2] = -np.cos(pseudo_phi)
    rotation_3[2][1] = np.cos(pseudo_phi)
    rotation_3[2][2] = np.sin(pseudo_phi)
    
    return rotation_3 @ double_rotation

In [5]:
def n_sigma_clipping(max_longitude_rad: float, longitudes_rad: np.ndarray, values: np.ndarray,
                     w: float, max_value: float, min_value: float, n: float) -> tuple:
    """Return the list of longitudes (in degrees) between -max_longitude_rad and max_longitude_rad where 
    there are stars within +/- w degrees and their angular moment is between min_value and max_value and the 
    list of average angular momentum in each of those ranges and the list of standard deviations in angular 
    momentum in each of those ranges, given a star cluster's longitudes_rad and values, both recalculated after 
    excluding values greater than n original standard deviations.
    """

    degs = []
    avg_delta_Lz = []
    std_delta_Lz = []

    for deg in np.arange(-max_longitude_rad * u.rad.to(u.deg), max_longitude_rad * u.rad.to(u.deg) + 1):

        delta_Lz_list = []

        for star in range(len(longitudes_rad)):

            if deg - w <= longitudes_rad[star] * u.rad.to(u.deg) < deg + w\
            and min_value <= values[star] <= max_value:
                delta_Lz_list.append(values[star])

        if delta_Lz_list != []:

            mean = np.mean(delta_Lz_list)
            sigma = np.std(delta_Lz_list)
            
            new_delta_Lz_list = []
            
            for v in delta_Lz_list:
                if abs(v - mean) <= n * sigma:
                    new_delta_Lz_list.append(v)
                    
            if new_delta_Lz_list != []:
                degs.append(deg)
                avg_delta_Lz.append(np.mean(new_delta_Lz_list))
                std_delta_Lz.append(np.std(new_delta_Lz_list))
            
    return(np.array(degs), avg_delta_Lz, std_delta_Lz)

In [6]:
# begin copied from previous notebook

In [7]:
dir = '/data/vxtest/519'

d = {'1330': {}, '1300': {}, '1200': {}, '1100': {}, '1000': {}}

In [8]:
for t in d:

    filename = dir + '/snapshot_' + t
    haloname = dir + '/halos/h' + t + '.z0.000.AHF_halos'

    dh = np.loadtxt(haloname)
    d[t]['xh'] = dh[:,5][0]
    d[t]['yh'] = dh[:,6][0]
    d[t]['zh'] = dh[:,7][0]
    d[t]['vxh'] = dh[:,8][0]
    d[t]['vyh'] = dh[:,9][0]
    d[t]['vzh'] = dh[:,10][0]

    hd = np.fromfile(filename, dtype='int32', count=7)
    d[t]['nd'] = hd[2]
    d[t]['ns'] = hd[5]
    d[t]['nc'] = hd[3]
    d[t]['nb'] = hd[4]
    head = np.fromfile(filename, dtype='double', count=7, offset=4+6*4)
    d[t]['time'] = head[6]
    
    start_d = 0
    start_c = d[t]['nd']
    start_b = d[t]['nd'] + d[t]['nc']
    start_s = d[t]['nd'] + d[t]['nc'] + d[t]['nb']
    block = d[t]['nd'] + d[t]['nc'] + d[t]['ns'] + d[t]['nb']

    if d[t]['nb'] > 0:
        skipx = 256 + 3*4 + start_b*3*8
        skipv = 256 + 5*4 + (block*3 + start_b*3)*8
        d[t]['rc'] = np.fromfile(filename,dtype='double',count=3,offset=skipx)
        d[t]['vc'] = np.fromfile(filename,dtype='double',count=3,offset=skipv)
        d[t]['xh'] = d[t]['rc'][0]
        d[t]['yh'] = d[t]['rc'][1]
        d[t]['zh'] = d[t]['rc'][2]
        d[t]['vxh'] = d[t]['vc'][0]
        d[t]['vyh'] = d[t]['vc'][1]
        d[t]['vzh'] = d[t]['vc'][2]
    
    skipx = 256 + 3*4 + start_c*3*8
    skipv = 256 + 5*4 + (block*3 + start_c*3)*8
    skipi = 256 + 7*4 + block*3*2*8 + start_c*4
    d[t]['rc'] = np.fromfile(filename, dtype='double', count=3*d[t]['nc'], offset=skipx)
    d[t]['vc'] = np.fromfile(filename, dtype='double', count=3*d[t]['nc'], offset=skipv)
    d[t]['idc'] = np.fromfile(filename, dtype='uint32', count=d[t]['nc'], offset=skipi)
    d[t]['jxc'] = d[t]['rc'][0::3] - d[t]['xh']
    d[t]['jyc'] = d[t]['rc'][1::3] - d[t]['yh']
    d[t]['jzc'] = d[t]['rc'][2::3] - d[t]['zh']

    d[t]['jvxc'] = d[t]['vc'][0::3] - d[t]['vxh']
    d[t]['jvyc'] = d[t]['vc'][1::3] - d[t]['vyh']
    d[t]['jvzc'] = d[t]['vc'][2::3] - d[t]['vzh']

    d[t]['idc'] -= d[t]['nd']
    ki = np.zeros(d[t]['nc'], dtype='int32')
    for j in range(d[t]['nc']):
        ix = d[t]['idc'][j]
        ki[ix] = j

    d[t]['xc'] = d[t]['jxc'][ki]
    d[t]['yc'] = d[t]['jyc'][ki]
    d[t]['zc'] = d[t]['jzc'][ki]

    d[t]['vxc'] = d[t]['jvxc'][ki]
    d[t]['vyc'] = d[t]['jvyc'][ki]
    d[t]['vzc'] = d[t]['jvzc'][ki]

    d[t]['rh3'] = [d[t]['xh'], d[t]['yh'], d[t]['zh']]
    d[t]['vh3'] = [d[t]['vxh'], d[t]['vyh'], d[t]['vzh']]
    
    #centers unsorted
    d[t]['jrc3'] = np.reshape(d[t]['rc'], (d[t]['nc'], 3)) - d[t]['rh3']
    d[t]['jvc3'] = np.reshape(d[t]['vc'],(d[t]['nc'], 3)) - d[t]['vh3']
 
    d[t]['rc3'] = d[t]['jrc3'][ki]
    d[t]['vc3'] = d[t]['jvc3'][ki]
 
    skipx = 256 + 3*4 + start_s*3*8
    skipv = 256 + 5*4 + block*3*8 + start_s*3*8 #skip over r, and dm+c v's
    skipi = 256 + 7*4 + block*3*2*8 + start_s*4 #skip over r and v and dm+c id
    skipc = 256 + 9*4 + block*(2*3*8+4) + start_s*4 #skip over r, v and id and dm+c ic

    d[t]['rs'] = np.fromfile(filename, dtype='double', count=3*d[t]['ns'], offset=skipx)
    d[t]['vs'] = np.fromfile(filename, dtype='double', count=3*d[t]['ns'], offset=skipv)
    d[t]['ids'] = np.fromfile(filename, dtype='uint32', count=d[t]['ns'], offset=skipi)
    d[t]['jics'] = np.fromfile(filename,dtype='uint32',count=d[t]['ns'],offset=skipc)

    d[t]['jrs3'] = np.reshape(d[t]['rs'], (d[t]['ns'], 3)) - d[t]['rh3']
    d[t]['jvs3'] = np.reshape(d[t]['vs'], (d[t]['ns'], 3)) - d[t]['vh3']
 
    d[t]['ids'] -= d[t]['nd'] + d[t]['nc'] + d[t]['nb']
    ki = np.zeros(d[t]['ns'], dtype='int32')
    for j in range(d[t]['ns']):
        ix = d[t]['ids'][j]
        ki[ix] = j

    d[t]['rs3'] = d[t]['jrs3'][ki]
    d[t]['vs3'] = d[t]['jvs3'][ki]
    d[t]['ics'] = d[t]['jics'][ki]

    # there is a change in shape vs the previous method;  
    # adapt to select the right array axis instead of skip by 3 along a single 1-d
    
    d[t]['xs'] = d[t]['rs3'][:,0]
    d[t]['ys'] = d[t]['rs3'][:,1]
    d[t]['zs'] = d[t]['rs3'][:,2]
    d[t]['vxs'] = d[t]['vs3'][:,0]
    d[t]['vys'] = d[t]['vs3'][:,1]
    d[t]['vzs'] = d[t]['vs3'][:,2]

#    print(len(d[t]['xs']), len(d[t]['ys']), len(d[t]['zs']))
    
    # distance from centre of galaxy
    d[t]['ds'] = np.sqrt(d[t]['xs'] ** 2 + d[t]['ys'] ** 2 + d[t]['zs'] ** 2)

In [9]:
# end copied from previous notebook

In [10]:
zoom = 4
dg = 1/90

lon_max = np.pi / zoom
lat_max = np.pi / 3 / zoom

ng_lon = int(2 * lon_max / dg + 1) 
ng_lat = int(2 * lat_max / dg + 1)

lon = np.linspace(-lon_max, lon_max, ng_lon + 1) 
lat = np.linspace(-lat_max, lat_max, ng_lat + 1) 
#lon_v, lat_v = np.meshgrid(lon, lat)

In [11]:
d_max = 100
d_min = 0
ng_d = int((d_max - d_min) * 1 + 1)

d_linspace = np.linspace(d_min, d_max, ng_d + 1) 

In [12]:
lon_v1, lat_v = np.meshgrid(lon, lat)
lon_v2, d_v = np.meshgrid(lon, d_linspace)   

In [13]:
L_x_max = 500
L_x_min = -500
ng_L_x = 100

L_x_linspace = np.linspace(L_x_min, L_x_max, ng_L_x + 1) 
lon_v3, L_x_v = np.meshgrid(lon, L_x_linspace)

In [14]:
L_y_max = 500
L_y_min = -500
ng_L_y = 100

L_y_linspace = np.linspace(L_y_min, L_y_max, ng_L_y + 1) 
lon_v4, L_y_v = np.meshgrid(lon, L_y_linspace)

In [15]:
L_z_max = 500
L_z_min = -500
ng_L_z = 100

L_z_linspace = np.linspace(L_z_min, L_z_max, ng_L_z + 1) 
lon_v5, L_z_v = np.meshgrid(lon, L_z_linspace)

In [16]:
lon_range_deg = np.arange(-40, 50, 10)
lon_range_deg_str = []

for deg in lon_range_deg:
    s = str(deg) + r'$^{\circ}$'
    lon_range_deg_str.append(s)
    
    
lat_range_deg = np.arange(-15, 20, 5)
lat_range_deg_str = []

for deg in lat_range_deg:
    s = str(deg) + r'$^{\circ}$'
    lat_range_deg_str.append(s)

In [17]:
# Zoomed in

In [ ]:
cluster = 208
chronological = True

fix = True
    
timestamp_list = []
stars_longitudes_list = []
stars_latitudes_list = []
stars_rho_list = []

Ds1_list = []
Ds2_list = []
Ds3_list = []
Ds4_list = []
Ds5_list = []

degs_x_list = []
degs_y_list = []
degs_z_list = []
avg_delta_Lx_list = []
avg_delta_Ly_list = []
avg_delta_Lz_list = []
std_delta_Lx_list = []
std_delta_Ly_list = []
std_delta_Lz_list = []

for t in sorted(d.keys(), reverse=not chronological):

    select = (d[t]['ics'] == cluster)

    cluster_x = d[t]['xc'][cluster]
    cluster_y = d[t]['yc'][cluster]
    cluster_z = d[t]['zc'][cluster]
    cluster_r = np.array([cluster_x, cluster_y, cluster_z])

    cluster_vx = d[t]['vxc'][cluster]
    cluster_vy = d[t]['vyc'][cluster]
    cluster_vz = d[t]['vzc'][cluster]
    cluster_v = np.array([cluster_vx, cluster_vy, cluster_vz])

    cluster_L = np.cross(cluster_r, cluster_v)

    transform = build_triple_rotation_matrix(ref_r=cluster_r, ref_L=cluster_L)

    new_cluster_r = transform @ cluster_r
    new_cluster_L = transform @ cluster_L

    new_cluster_Lx = new_cluster_L[0]
    new_cluster_Ly = new_cluster_L[1]
    new_cluster_Lz = new_cluster_L[2]

    ####
    stars_d = d[t]['ds'][select]
    ####

    stars_x = d[t]['xs'][select]
    stars_y = d[t]['ys'][select]
    stars_z = d[t]['zs'][select]

    stars_vx = d[t]['vxs'][select]
    stars_vy = d[t]['vys'][select]
    stars_vz = d[t]['vzs'][select]
    
    timestamp = d[t]['time']

    new_stars_Lx = []
    new_stars_Ly = []
    new_stars_Lz = []

    new_stars_x = []
    new_stars_y = []
    new_stars_z = []

    for i in range(len(stars_x)):

        star_x = stars_x[i]
        star_y = stars_y[i]
        star_z = stars_z[i]
        star_r = [star_x, star_y, star_z]

        star_vx = stars_vx[i]
        star_vy = stars_vy[i]
        star_vz = stars_vz[i]
        star_v = [star_vx, star_vy, star_vz]

        star_L = np.cross(star_r, star_v)
        new_star_L = transform @ star_L

        new_stars_Lx.append(new_star_L[0])
        new_stars_Ly.append(new_star_L[1])
        new_stars_Lz.append(new_star_L[2])

        new_star_r = transform @ star_r

        new_stars_x.append(new_star_r[0])
        new_stars_y.append(new_star_r[1])
        new_stars_z.append(new_star_r[2])

    new_stars_Lx = np.array(new_stars_Lx)
    new_stars_Ly = np.array(new_stars_Ly)
    new_stars_Lz = np.array(new_stars_Lz)

    new_stars_x = np.array(new_stars_x)
    new_stars_y = np.array(new_stars_y)
    new_stars_z = np.array(new_stars_z)

    new_stars_delta_Lx = new_stars_Lx - new_cluster_Lx
    new_stars_delta_Ly = new_stars_Ly - new_cluster_Ly
    new_stars_delta_Lz = new_stars_Lz - new_cluster_Lz

    new_stars_L_perp = np.sqrt((new_stars_delta_Lx) ** 2 + (new_stars_delta_Ly) ** 2)

    new_stars_rho = np.sqrt(new_stars_x ** 2 + new_stars_y ** 2 + new_stars_z ** 2)

    # longs and lats formulae taken from previous notebook
    stars_longitudes = np.arctan2(new_stars_y, new_stars_x)
    stars_latitudes = np.arcsin(new_stars_z / new_stars_rho)

    if fix:
        stars_longitudes_fixed = stars_longitudes
        t_fixed = timestamp
        fix = False

    D = np.zeros((ng_lat + 1, ng_lon + 1))
    for star in range(len(stars_longitudes)):
        if (-lon_max <= stars_longitudes[star] <= lon_max) and (-lat_max <= stars_latitudes[star] <= lat_max):
            ix = int(round((stars_longitudes[star] + lon_max) / dg)) 
            iy = int(round((stars_latitudes[star] + lat_max) / dg)) 
            D[iy, ix] += 1
    Ds1 = np.sqrt(D)

    D = np.zeros((ng_d + 1, ng_lon + 1))
    for star in range(len(stars_longitudes)):
        if (-lon_max <= stars_longitudes[star] <= lon_max) \
    and (d_min <= stars_d[star] <= d_max):
            ix = int(round((stars_longitudes[star] + lon_max) / dg)) 
            iy = int(round((stars_d[star] + -d_min) * 1)) 
            D[iy, ix] += 1
    Ds2 = np.sqrt(D)

    D = np.zeros((ng_L_x + 1, ng_lon + 1))
    for star in range(len(stars_longitudes_fixed)):
        if (-lon_max <= stars_longitudes_fixed[star] <= lon_max) \
    and (L_x_min <= new_stars_delta_Lx[star] <= L_x_max):
            ix = int(round((stars_longitudes_fixed[star] + lon_max) / dg)) 
            iy = int(round((new_stars_delta_Lx[star] + -L_x_min) / 10)) 
            D[iy, ix] += 1
    Ds3 = np.sqrt(D)
    
    D = np.zeros((ng_L_y + 1, ng_lon + 1))
    for star in range(len(stars_longitudes_fixed)):
        if (-lon_max <= stars_longitudes_fixed[star] <= lon_max) \
        and (L_y_min <= new_stars_delta_Ly[star] <= L_y_max):
            ix = int(round((stars_longitudes_fixed[star] + lon_max) / dg)) 
            iy = int(round((new_stars_delta_Ly[star] + -L_y_min) / 10)) 
            D[iy, ix] += 1
    Ds4 = np.sqrt(D)

    D = np.zeros((ng_L_z + 1, ng_lon + 1))
    for star in range(len(stars_longitudes_fixed)):
        if (-lon_max <= stars_longitudes_fixed[star] <= lon_max) \
    and (L_z_min <= new_stars_delta_Lz[star] <= L_z_max):
            ix = int(round((stars_longitudes_fixed[star] + lon_max) / dg)) 
            iy = int(round((new_stars_delta_Lz[star] + -L_z_min) / 10)) 
            D[iy, ix] += 1
    Ds5 = np.sqrt(D)
    
    (degs_x, avg_delta_Lx, std_delta_Lx) = n_sigma_clipping(max_longitude_rad=lon_max, 
                                                      longitudes_rad=stars_longitudes_fixed, 
                                                      values=new_stars_delta_Lx,
                                                      w=2, max_value=L_x_max, min_value=L_x_min, n=2)
    
    (degs_y, avg_delta_Ly, std_delta_Ly) = n_sigma_clipping(max_longitude_rad=lon_max, 
                                                      longitudes_rad=stars_longitudes_fixed, 
                                                      values=new_stars_delta_Ly,
                                                      w=2, max_value=L_y_max, min_value=L_y_min, n=2)
 
    (degs_z, avg_delta_Lz, std_delta_Lz) = n_sigma_clipping(max_longitude_rad=lon_max, 
                                                      longitudes_rad=stars_longitudes_fixed, 
                                                      values=new_stars_delta_Lz,
                                                      w=2, max_value=L_z_max, min_value=L_z_min, n=2)

    
    
    timestamp_list.append(timestamp)
    stars_longitudes_list.append(stars_longitudes)
    stars_latitudes_list.append(stars_latitudes)
    stars_rho_list.append(new_stars_rho)
    
    Ds1_list.append(Ds1)
    Ds2_list.append(Ds2)
    Ds3_list.append(Ds3)
    Ds4_list.append(Ds4)
    Ds5_list.append(Ds5)

    degs_x_list.append(degs_x)
    degs_y_list.append(degs_y)
    degs_z_list.append(degs_z)
    avg_delta_Lx_list.append(avg_delta_Lx)
    avg_delta_Ly_list.append(avg_delta_Ly)
    avg_delta_Lz_list.append(avg_delta_Lz)
    std_delta_Lx_list.append(std_delta_Lx)
    std_delta_Ly_list.append(std_delta_Ly)
    std_delta_Lz_list.append(std_delta_Lz)

In [ ]:
length = len(stars_longitudes_list)

plt.figure(figsize=(15, 3.5 * length))
plt.rc('font', size=14)

for j in range(length):

    plt.subplot(length, 2, 2 * j + 1)
    #plt.pcolormesh(lon_v1, lat_v, Ds1_list[j], cmap='binary', vmin=1, vmax=2)
    plt.scatter(stars_longitudes_list[j], stars_latitudes_list[j], s=8, c='royalblue', alpha=0.5)

    plt.xlim(-lon_max, lon_max)
    plt.ylim(-17 * u.deg.to(u.rad), 17 * u.deg.to(u.rad))

    if j == length - 1:
        plt.xticks(ticks=lon_range_deg * u.deg.to(u.rad), labels=lon_range_deg_str)
        plt.xlabel('Galactic longitude\n(a)')
    else:
        plt.xticks(ticks=lon_range_deg * u.deg.to(u.rad), labels=[])
    
    plt.yticks(ticks=lat_range_deg * u.deg.to(u.rad), labels=lat_range_deg_str)

    plt.ylabel('Galactic latitude')
    plt.grid()
#plt.title('Time = %.2f Gy' % timestamp_list[j], size=14, loc='right')


    plt.subplot(length, 2,  2 * j + 2)
    #plt.pcolormesh(lon_v2, d_v, Ds2_list[j], cmap='binary', vmin=0, vmax=2)
    plt.scatter(stars_longitudes_list[j], stars_rho_list[j], s=8, c='royalblue', alpha=0.5)

    if j == length - 1:
        plt.xticks(ticks=lon_range_deg * u.deg.to(u.rad), labels=lon_range_deg_str)
        plt.xlabel('Galactic longitude\n(b)')
    else:
        plt.xticks(ticks=lon_range_deg * u.deg.to(u.rad), labels=[])
    
    plt.xlim(-lon_max, lon_max)
    
    plt.ylabel('Distance  [kpc]')
    plt.grid()
    plt.title('Time = %.2f Gy' % timestamp_list[j], size=14, loc='right')
    
#plt.suptitle('Morphology of cluster 268  (total number of stars: 10929)', fontweight='bold')

plt.subplots_adjust(hspace=0.1)
plt.savefig('Dispersion_evolution.pdf', format='pdf', bbox_inches='tight')
plt.show()

In [ ]:
length = len(stars_longitudes_list)

plt.figure(figsize=(15, 5 * length))
plt.rc('font', size=14)

for j in range(length):

    plt.subplot(length, 3, 3 * j + 1)
    plt.pcolormesh(lon_v3, L_x_v, Ds3_list[j], cmap='binary', vmin=0, vmax=2)

    if j == length - 1:
        plt.xticks(ticks=lon_range_deg * u.deg.to(u.rad), labels=lon_range_deg_str)
        plt.xlabel('Galactic longitude at $t$ = %.1f Gy\n(a)' % t_fixed)
    else:
        plt.xticks(ticks=lon_range_deg * u.deg.to(u.rad), labels=[])
    
    plt.ylabel(r'$\Delta \, L_x$  [kpc$\cdot$km/s]', size=12)
    plt.yticks(size=12)
    plt.grid()
#    plt.title('Time = %.2f Gy' % timestamp_list[j], size=14, loc='right')
    

    plt.subplot(length, 3,  3 * j + 2)
    plt.pcolormesh(lon_v4, L_y_v, Ds4_list[j], cmap='binary', vmin=0, vmax=2)

    if j == length - 1:
        plt.xticks(ticks=lon_range_deg * u.deg.to(u.rad), labels=lon_range_deg_str)
        plt.xlabel('Galactic longitude at $t$ = %.1f Gy\n(b)' % t_fixed)
    else:
        plt.xticks(ticks=lon_range_deg * u.deg.to(u.rad), labels=[])
    
    plt.ylabel(r'$\Delta \, L_y$  [kpc$\cdot$km/s]', size=12)
    plt.yticks(size=12)
    plt.grid()
#    plt.title('Time = %.2f Gy' % timestamp_list[j], size=14, loc='right')

    
    plt.subplot(length, 3,  3 * j + 3)
    plt.pcolormesh(lon_v5, L_z_v, Ds5_list[j], cmap='binary', vmin=0, vmax=2)

    if j == length - 1:
        plt.xticks(ticks=lon_range_deg * u.deg.to(u.rad), labels=lon_range_deg_str)
        plt.xlabel('Galactic longitude at $t$ = %.1f Gy\n(c)' % t_fixed)
    else:
        plt.xticks(ticks=lon_range_deg * u.deg.to(u.rad), labels=[])
    
    plt.ylabel(r'$\Delta \, L_z$  [kpc$\cdot$km/s]', size=12)
    plt.yticks(size=12)
    plt.grid()
    plt.title('Time = %.2f Gy' % timestamp_list[j], size=14, loc='right')


plt.subplots_adjust(hspace=0.1, wspace=0.3)
plt.show()

In [ ]:
length = len(stars_longitudes_list)

plt.figure(figsize=(24, 4 * length))
plt.rc('font', size=14)

for j in range(length):

    plt.subplot(length, 3, 3 * j + 1)
    plt.pcolormesh(lon_v3, L_x_v, Ds3_list[j], cmap='binary', vmin=0, vmax=2)

    if j == length - 1:
        plt.xticks(ticks=lon_range_deg * u.deg.to(u.rad), labels=lon_range_deg_str)
        plt.xlabel('Galactic longitude at $t$ = %.1f Gy\n(a)' % t_fixed)
    else:
        plt.xticks(ticks=lon_range_deg * u.deg.to(u.rad), labels=[])
    
    plt.plot(degs_x_list[j] * u.deg.to(u.rad), std_delta_Lx_list[j], '-', color='firebrick',
                label=r'$\sigma_{\Delta \, L_x}$')
    plt.plot(degs_x_list[j] * u.deg.to(u.rad), avg_delta_Lx_list[j], '-', color='royalblue', 
                label=r'$<\Delta \, L_x>$')
    
    plt.legend()
    plt.ylabel(r'$\Delta \, L_x$  [kpc$\cdot$km/s]', size=12)
    plt.yticks(size=12)
    plt.grid()
#    plt.title('Time = %.2f Gy' % timestamp_list[j], size=14, loc='right')
    

    plt.subplot(length, 3,  3 * j + 2)
    plt.pcolormesh(lon_v4, L_y_v, Ds4_list[j], cmap='binary', vmin=0, vmax=2)

    if j == length - 1:
        plt.xticks(ticks=lon_range_deg * u.deg.to(u.rad), labels=lon_range_deg_str)
        plt.xlabel('Galactic longitude at $t$ = %.1f Gy\n(b)' % t_fixed)
    else:
        plt.xticks(ticks=lon_range_deg * u.deg.to(u.rad), labels=[])
    
    plt.plot(degs_y_list[j] * u.deg.to(u.rad), std_delta_Ly_list[j], '-', color='firebrick',
                label=r'$\sigma_{\Delta \, L_y}$')
    plt.plot(degs_y_list[j] * u.deg.to(u.rad), avg_delta_Ly_list[j], '-', color='royalblue', 
                label=r'$<\Delta \, L_y>$')
    
    plt.legend()
    plt.ylabel(r'$\Delta \, L_y$  [kpc$\cdot$km/s]', size=12)
    plt.yticks(size=12)
    plt.grid()
#    plt.title('Time = %.2f Gy' % timestamp_list[j], size=14, loc='right')

    
    plt.subplot(length, 3,  3 * j + 3)
    plt.pcolormesh(lon_v5, L_z_v, Ds5_list[j], cmap='binary', vmin=0, vmax=2)

    if j == length - 1:
        plt.xticks(ticks=lon_range_deg * u.deg.to(u.rad), labels=lon_range_deg_str)
        plt.xlabel('Galactic longitude at $t$ = %.1f Gy\n(c)' % t_fixed)
    else:
        plt.xticks(ticks=lon_range_deg * u.deg.to(u.rad), labels=[])
    
    plt.plot(degs_z_list[j] * u.deg.to(u.rad), std_delta_Lz_list[j], '-', color='firebrick',
                label=r'$\sigma_{\Delta \, L_z}$')
    plt.plot(degs_z_list[j] * u.deg.to(u.rad), avg_delta_Lz_list[j], '-', color='royalblue', 
                label=r'$<\Delta \, L_z>$')
    
    plt.legend()
    plt.ylabel(r'$\Delta \, L_z$  [kpc$\cdot$km/s]', size=12)
    plt.yticks(size=12)
    plt.grid()
    plt.title('Time = %.2f Gy' % timestamp_list[j], size=14, loc='right')


plt.subplots_adjust(hspace=0.1, wspace=0.3)
plt.savefig('Angmome_evolution.pdf', format='pdf', bbox_inches='tight')
plt.show()